In [ ]:
import numpy as np
from numpy.linalg import norm, inv
import pandas as pd

In [ ]:
# define the function of interest
def f(x):
    return (5 * x[0] - x[1])**4 + (x[0] - 2)**2 + x[0] - 2*x[1] + 12

# define the gradient of the said function
def gradient(x):
    dx1 = 20 * ( 5 * x[0] - x[1])**3 + 2 * (x[0] - 2) + 1
    dx2 = -4 * ( 5 * x[0] - x[1])**3 - 2
    return np.array([dx1, dx2])

# to be used for line search
# parameter set  (ϵ2, a, b) = (0.005,-100, 100)
def Golden_Section(f, a=-100, b=100, epsilon=0.005):
    gamma = (1 + np.sqrt(5))/2
    c = 1/gamma

    x = b - c*(b-a) # defining the x_0
    y = a + c*(b-a) # defining the y_0

    fx = f(x)       # defining the f(x_0)
    fy = f(y)       # defining the f(y_0)

    while abs(b-a) > epsilon:
        if fx > fy:
            a = x
            x = y
            fx = fy

            y = a + c*(b-a)
            fy = f(y)

        else:
            b = y
            y = x
            fy = fx

            x = b - c*(b-a)
            fx = f(x)

    return x

In [ ]:
def BFGS(function, gradient, x0, epsilon=1e-6):

    # lists to keep track of the variables
    x_list = []
    f_list = []
    d_list = []
    alpha_list = []
    xnew_list = []

    f = function
    grad_f = gradient

    n = len(x0)
    x = np.array(x0, dtype=float)
    H = np.eye(n)  # H0 = I
    g = grad_f(x)

    # do until || ∇f(x) || <= ϵ1
    while norm(g) > epsilon:
        g = grad_f(x)
        x_list.append(x)
        f_list.append(f(x))

        # Search direction
        d = -H.dot(g)
        d_list.append(d)

        # Line search
        def f_line(a):
            return f(x + a * d)
        alpha = Golden_Section(f_line)
        alpha_list.append(alpha)

        # Update position
        x_new = x + alpha * d
        xnew_list.append(x_new)

        # Compute difference vectors
        s = x_new - x
        y = grad_f(x_new) - g

        r = 1.0 / y.dot(s)
        I = np.eye(n)

        # H(k+1)
        H = (I - r * np.outer(s, y)).dot(H).dot(I - r * np.outer(y, s)) + r * np.outer(s, s)

        x = x_new

    return pd.DataFrame({'x(k)': x_list, 'f(x(k))': f_list, 'd(k)': d_list, 'alpha(k)': alpha_list, 'x(k+1)': xnew_list}).rename_axis('k')

$$
(\epsilon_1, x^{(0)}) = (1e-6, (0, 0))
$$

In [ ]:
x0 = np.array([0.0, 0.0])
BFGS(f, gradient, x0)

,x(k),f(x(k)),d(k),alpha(k),x(k+1)
k,,,,,
0,"[0.0, 0.0]",16.000000,"[3.0, 2.0]",0.046871,"[0.14061287713497048, 0.09374191808998032]"
1,"[0.14061287713497048, 0.09374191808998032]",15.548294,"[0.916645841403258, 4.847769507273998]",6.432373,"[6.036821235065612, 31.276405707141738]"
2,"[6.036821235065612, 31.276405707141738]",-26.796533,"[1.0358127262612682, 3.1816373004794385]",0.170177,"[6.213092539551373, 31.817846590360432]"
3,"[6.213092539551373, 31.817846590360432]",-27.352004,"[0.3134900823680328, 1.5988262410598655]",0.929757,"[6.504562147282392, 33.30436652678601]"
4,"[6.504562147282392, 33.30436652678601]",-27.439978,"[-0.003180051117196654, -0.0070058333385178955]",1.367711,"[6.500212757928666, 33.29478457485708]"
5,"[6.500212757928666, 33.29478457485708]",-27.440551,"[-0.0002131428000711567, -0.0010863233732660497]",0.997056,"[6.500000242586265, 33.29370144944281]"
6,"[6.500000242586265, 33.29370144944281]",-27.440551,"[-2.4423554465828717e-07, -9.271999171959466e-07]",0.927828,"[6.500000015977674, 33.293700589160714]"
7,"[6.500000015977674, 33.293700589160714]",-27.440551,"[-1.597890399410746e-08, -6.318302527153212e-08]",-1.317491,"[6.500000037029732, 33.293700672403766]"


$$
(\epsilon_1, x^{(0)}) = (5e-8, (3, 3))
$$

In [ ]:
x0 = np.array([3.0, 3.0])
BFGS(f, gradient, x0, epsilon=5e-8)

,x(k),f(x(k)),d(k),alpha(k),x(k+1)
k,,,,,
0,"[3.0, 3.0]",20746.000000,"[-34563.0, 6914.0]",0.000228,"[-4.869463848537726, 4.57421152818881]"
1,"[-4.869463848537726, 4.57421152818881]",699702.041862,"[7.654370938600062, 0.07869798711405858]",0.766787,"[0.9998083005050145, 4.634556121841172]"
2,"[0.9998083005050145, 4.634556121841172]",4.748729,"[0.4230833220588754, 2.1154646278746627]",12.999854,"[6.499829823673218, 32.13528795922743]"
3,"[6.499829823673218, 32.13528795922743]",-25.504749,"[0.00029454558866762735, 0.0021363863625945756]",99.998071,"[6.529283814368863, 32.34892247445072]"
4,"[6.529283814368863, 32.34892247445072]",-25.646316,"[0.10170611553558007, 2.019290896337639]",0.720144,"[6.602526835593074, 33.80310214889294]"
5,"[6.602526835593074, 33.80310214889294]",-27.430000,"[-0.0829084494170663, -0.4031994334493562]",1.172055,"[6.505353538935339, 33.33053007168871]"
6,"[6.505353538935339, 33.33053007168871]",-27.440136,"[-0.020041710578216267, -0.1384717721318538]",0.262727,"[6.500088049360794, 33.29414986004542]"
7,"[6.500088049360794, 33.29414986004542]",-27.440551,"[-8.807072234699083e-05, -0.0004493237140343909]",1.000177,"[6.499999963022602, 33.29370045666167]"
8,"[6.499999963022602, 33.29370045666167]",-27.440551,"[3.644502610316778e-08, 6.804475842743654e-08]",0.681216,"[6.49999998784955, 33.29370050301487]"


In [ ]:
BFGS(f, gradient, x0).to_latex()

'\\begin{tabular}{llrlrl}\n\\toprule\n & x(k) & f(x(k)) & d(k) & alpha(k) & x(k+1) \\\\\n\\midrule\n0 & [3. 3.] & 20746.000000 & [-34563.   6914.] & 0.000228 & [-4.86946385  4.57421153] \\\\\n1 & [-4.86946385  4.57421153] & 699702.041862 & [7.65437094 0.07869799] & 0.766787 & [0.9998083  4.63455612] \\\\\n2 & [0.9998083  4.63455612] & 4.748729 & [0.42308332 2.11546463] & 12.999854 & [ 6.49982982 32.13528796] \\\\\n3 & [ 6.49982982 32.13528796] & -25.504749 & [0.00029455 0.00213639] & 99.998071 & [ 6.52928381 32.34892247] \\\\\n4 & [ 6.52928381 32.34892247] & -25.646316 & [0.10170612 2.0192909 ] & 0.720144 & [ 6.60252684 33.80310215] \\\\\n5 & [ 6.60252684 33.80310215] & -27.430000 & [-0.08290845 -0.40319943] & 1.172055 & [ 6.50535354 33.33053007] \\\\\n6 & [ 6.50535354 33.33053007] & -27.440136 & [-0.02004171 -0.13847177] & 0.262727 & [ 6.50008805 33.29414986] \\\\\n7 & [ 6.50008805 33.29414986] & -27.440551 & [-8.80707223e-05 -4.49323714e-04] & 1.000177 & [ 6.49999996 33.29370046] \\\